In [1]:
from pymongo import MongoClient
from bson import ObjectId
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
client = MongoClient("mongodb://localhost:27017/")
	
db = client["lifesnaps"]
collection = db["fitbit"]

person = "621e2f9167b776a240011ccb"

query = {
    "id": ObjectId(person),
    "type": "steps" 
}

doc = collection.find(query)
for doc in collection.find(query):
    print(doc)

In [3]:
for doc in collection.find(query):
    recorded_time = doc["data"]["dateTime"]
    date = recorded_time.split("T")[0]
    time = recorded_time.split("T")[1]
    steps = doc["data"]["value"]
    print(f"Date: {date}, Time: {time}, Temperature: {steps}")


Date: 2021-10-23, Time: 11:28:00, Temperature: 0
Date: 2021-10-23, Time: 11:54:00, Temperature: 14
Date: 2021-10-23, Time: 11:55:00, Temperature: 0
Date: 2021-10-23, Time: 11:59:00, Temperature: 4
Date: 2021-10-23, Time: 12:03:00, Temperature: 27
Date: 2021-10-23, Time: 12:10:00, Temperature: 26
Date: 2021-10-23, Time: 12:16:00, Temperature: 0
Date: 2021-10-23, Time: 12:21:00, Temperature: 19
Date: 2021-10-23, Time: 12:44:00, Temperature: 23
Date: 2021-10-23, Time: 12:45:00, Temperature: 0
Date: 2021-10-23, Time: 12:54:00, Temperature: 0
Date: 2021-10-23, Time: 12:56:00, Temperature: 0
Date: 2021-10-23, Time: 13:06:00, Temperature: 0
Date: 2021-10-23, Time: 13:20:00, Temperature: 81
Date: 2021-10-23, Time: 13:37:00, Temperature: 28
Date: 2021-10-23, Time: 13:43:00, Temperature: 42
Date: 2021-10-23, Time: 13:45:00, Temperature: 47
Date: 2021-10-23, Time: 13:53:00, Temperature: 30
Date: 2021-10-23, Time: 13:55:00, Temperature: 0
Date: 2021-10-23, Time: 13:57:00, Temperature: 0
Date: 2021

In [4]:
records = []
pipeline = [
    { "$match": query },
    { "$project": {
        "date": { "$substrBytes": [ "$data.dateTime", 0, 10 ] },
        "valueNum": { "$toInt": "$data.value" }
    }},
    { "$group": {
        "_id": "$date",
        "totalSteps": { "$sum": "$valueNum" }
    }},
    { "$sort": { "_id": 1 } }
]

for doc in collection.aggregate(pipeline):
    #print(f"{doc['_id']}: {doc['totalSteps']} steps")
    records.append({"date": doc["_id"], "steps": doc["totalSteps"]})
    
df = pd.DataFrame(records)

In [5]:
df

,date,steps
0,2021-10-23,6873
1,2021-10-24,5317
2,2021-10-25,6640
3,2021-10-26,6599
4,2021-10-27,7327
...,...,...
84,2022-01-15,7123
85,2022-01-16,3120
86,2022-01-17,7876
87,2022-01-18,4958


In [ ]:
df_day = df.loc['2021-10-23']

KeyError: '2021-10-23'